In [223]:
import cv2
import numpy as np

In [224]:
def binary_conversion(n):
    binary_number = ''
    while n > 0:
        binary_number = str(n % 2) + binary_number
        n //= 2
    return binary_number

In [225]:
def string_to_bin(message):
  binary_message = ""
  for char in message:
    # Convert each character to its 8-bit binary representation
    binary_char = bin(ord(char))[2:].zfill(8)  # Pad with zeros to 8 bits
    binary_message += binary_char
  # Append end-of-message marker (8 consecutive 0s)
  binary_message += '00000000'
  return binary_message

In [226]:
def embed_bit(channel, bit):
    # Embeds the bit into the channel by modifying the least significant bit
    if bit == '0':
        # If the bit to be embedded is 0, clear the least significant bit
        return channel & 0xFE
    else:
        # If the bit to be embedded is 1, set the least significant bit
        return channel | 0x01


In [227]:
def encrypt(cover_image, message):
    height, width, depth = cover_image.shape
    binary_message = string_to_bin(message)
    message_index = 0

    for row in range(height):
        for column in range(width):
            for channel in range(depth):
                if message_index < len(binary_message):
                    channel_value = cover_image[row, column, channel]
                    bit_to_embed = binary_message[message_index]
                    modified_channel = embed_bit(channel_value, bit_to_embed)
                    cover_image[row, column, channel] = modified_channel
                    message_index += 1
                else:
                    return cover_image
    return cover_image

In [228]:
def decrypt(stego_image):
  binary_message = ""
  height, width, depth = stego_image.shape
  message_index = 0

  for row in range(height):
    for column in range(width):
      for channel in range(depth):
        channel_value = stego_image[row, column, channel]
        extracted_LSB = channel_value & 0x01  # Extract LSB
        binary_message += str(extracted_LSB)
        message_index += 1

        # Check for end-of-message marker (8 consecutive 0s)
        if len(binary_message) >= 8 and binary_message[-8:] == '00000000':
          return binary_message[:-8]  # Return message excluding ending marker

  # If end-of-message marker not found, return entire message
  return binary_message

In [229]:
image = cv2.imread('images.jpeg')
message = "Hello this is a test message"

stego_image= encrypt(image,message)
stego_img_path= "Stego_image.png"
cv2.imwrite(stego_img_path,stego_image)

binary_message = decrypt(stego_image)

def binary_to_string(binary_message):
    message = ""
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i + 8]
        char = chr(int(byte, 2))
        message += char
    return message

# Convert binary message to string and print the original message
original_message = binary_to_string(binary_message)
print("Original message:", original_message)

Original message: Hello
